## Recommender System

In [290]:
# ---unzip the dataset using zipfile library
# import zipfile
# zip_ref=zipfile.ZipFile('dataset.csv.zip')
# zip_ref.extractall()
# zip_ref.close()

In [1]:
import pandas as pd
import numpy as np
import re

In [2]:
df=pd.read_csv('data/dataset.csv')

In [6]:
df[df['track_id']=="7ugjIppiHC3TgC9LuKbEwi"]

,Unnamed: 0,track_id,artists,album_name,track_name,popularity,duration_ms,explicit,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature,track_genre
61046,61046,7ugjIppiHC3TgC9LuKbEwi,=LOVE,手遅れcaution,「部活中に目が合うなって思ってたんだ」,33,283640,False,0.503,0.95,7,-1.62,1,0.0703,0.61,0.000014,0.112,0.576,159.022,4,j-idol
63891,63891,7ugjIppiHC3TgC9LuKbEwi,=LOVE,手遅れcaution,「部活中に目が合うなって思ってたんだ」,33,283640,False,0.503,0.95,7,-1.62,1,0.0703,0.61,0.000014,0.112,0.576,159.022,4,j-rock


In [7]:
df.head()

,Unnamed: 0,track_id,artists,album_name,track_name,popularity,duration_ms,explicit,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature,track_genre
0,0,5SuOikwiRyPMVoIQDJUgSV,Gen Hoshino,Comedy,Comedy,73,230666,False,0.676,0.4610,1,-6.746,0,0.1430,0.0322,0.000001,0.3580,0.715,87.917,4,acoustic
1,1,4qPNDBW1i3p13qLCt0Ki3A,Ben Woodward,Ghost (Acoustic),Ghost - Acoustic,55,149610,False,0.420,0.1660,1,-17.235,1,0.0763,0.9240,0.000006,0.1010,0.267,77.489,4,acoustic
2,2,1iJBSr7s7jYXzM8EGcbK5b,Ingrid Michaelson;ZAYN,To Begin Again,To Begin Again,57,210826,False,0.438,0.3590,0,-9.734,1,0.0557,0.2100,0.000000,0.1170,0.120,76.332,4,acoustic
3,3,6lfxq3CG4xtTiEg7opyCyx,Kina Grannis,Crazy Rich Asians (Original Motion Picture Sou...,Can't Help Falling In Love,71,201933,False,0.266,0.0596,0,-18.515,1,0.0363,0.9050,0.000071,0.1320,0.143,181.740,3,acoustic
4,4,5vjLSffimiIP26QG5WcN2K,Chord Overstreet,Hold On,Hold On,82,198853,False,0.618,0.4430,2,-9.681,1,0.0526,0.4690,0.000000,0.0829,0.167,119.949,4,acoustic


### Description of columns in the dataset
- **track_id(str)**: unique id for each track
- **artists(str)**: artists who performed by the track. More than one artist are seperated by ';'
- **album_name(str)**: name of the album to which the track belongs
- **track_name(str)**:name of the track
- **popularity(int)**: defined in the scale 1 to 100. According to the dataset it was based on algorithm which measured the     
                   popularity by measuring the no. of streams and how recent those streams are.
- **duration_ms()**: duration in millisec
- **explicit(bool)**: if the track has any explicit lyrics
- **danceability(float)**: how suitable a track is for dancing on the scale of 0 to 1 based on some of the factors such as tempo,beat strength, etc
- **energy(float)**: On a scale of 0 to 1, it is defined on the basis of feeling fast, noisy and loud
- **track_genre(str)**: Genre of the track     
There are some other factors which also describes the characteristics of the track

In [8]:
cols=df.columns
cols

Index(['Unnamed: 0', 'track_id', 'artists', 'album_name', 'track_name',
       'popularity', 'duration_ms', 'explicit', 'danceability', 'energy',
       'key', 'loudness', 'mode', 'speechiness', 'acousticness',
       'instrumentalness', 'liveness', 'valence', 'tempo', 'time_signature',
       'track_genre'],
      dtype='object')

In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 114000 entries, 0 to 113999
Data columns (total 21 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   Unnamed: 0        114000 non-null  int64  
 1   track_id          114000 non-null  object 
 2   artists           113999 non-null  object 
 3   album_name        113999 non-null  object 
 4   track_name        113999 non-null  object 
 5   popularity        114000 non-null  int64  
 6   duration_ms       114000 non-null  int64  
 7   explicit          114000 non-null  bool   
 8   danceability      114000 non-null  float64
 9   energy            114000 non-null  float64
 10  key               114000 non-null  int64  
 11  loudness          114000 non-null  float64
 12  mode              114000 non-null  int64  
 13  speechiness       114000 non-null  float64
 14  acousticness      114000 non-null  float64
 15  instrumentalness  114000 non-null  float64
 16  liveness          11

In [10]:
df[df.isnull().any(axis=1)]

,Unnamed: 0,track_id,artists,album_name,track_name,popularity,duration_ms,explicit,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature,track_genre
65900,65900,1kR4gIb7nGxHPI3D2ifs59,NaN,NaN,NaN,0,0,False,0.501,0.583,7,-9.46,0,0.0605,0.69,0.00396,0.0747,0.734,138.391,4,k-pop


In [11]:
df.dropna(axis=0,inplace=True)

In [12]:
df.isnull().sum()

Unnamed: 0          0
track_id            0
artists             0
album_name          0
track_name          0
popularity          0
duration_ms         0
explicit            0
danceability        0
energy              0
key                 0
loudness            0
mode                0
speechiness         0
acousticness        0
instrumentalness    0
liveness            0
valence             0
tempo               0
time_signature      0
track_genre         0
dtype: int64

In [13]:
df.duplicated().sum() # There are no duplicate rows present in this data

np.int64(0)

In [14]:
#But with genres being different same data is duplicated. For better results, they can be removed
groupby_data=df.groupby(['track_id']).size().reset_index().rename(columns={0:"Count"})
groupby_data=groupby_data[groupby_data.Count!=1]
groupby_data

,track_id,Count
6,001APMDOl3qtx1526T11n1,2
7,001YQlnDSduXd5LgBd66gT,2
12,003vvx7Niy0yvhvHt4a68B,3
14,004h8smbIoAkUNDJvVKwkG,2
23,006rHBBNLJMpQs8fRC2GDe,3
...,...,...
89717,7ztSVy67w9rXpKg5L2zN5l,2
89723,7zubR9uYAWjb5KPZTMm85e,4
89724,7zumacGldlmxpoP8bpaeLe,2
89726,7zv2vmZq8OjS54BxFzI2wM,2


In [15]:
df.track_id.nunique()

89740

In [16]:
df = df.drop_duplicates(subset='track_id').reset_index(drop=True)

In [17]:
df.reset_index(drop=True, inplace=True)

In [18]:
df.shape

(89740, 21)

In [19]:
tg=df['track_genre'].value_counts()
print(tg.head())
print(tg.tail())

track_genre
acoustic    1000
afrobeat     999
alt-rock     999
ambient      999
cantopop     999
Name: count, dtype: int64
track_genre
metal        232
punk         226
house        210
indie        134
reggaeton     74
Name: count, dtype: int64


In [20]:
pop=df['popularity'].value_counts()
print(pop.head())
print(pop.tail())

popularity
0     9447
21    2271
22    2258
20    2058
23    2038
Name: count, dtype: int64
popularity
98     2
94     2
95     2
100    1
99     1
Name: count, dtype: int64


In [21]:
df[df.popularity==100]

,Unnamed: 0,track_id,artists,album_name,track_name,popularity,duration_ms,explicit,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature,track_genre
18906,20001,3nqQXoyQOWXiESFLlDF1hG,Sam Smith;Kim Petras,Unholy (feat. Kim Petras),Unholy (feat. Kim Petras),100,156943,False,0.714,0.472,2,-7.375,1,0.0864,0.013,0.000005,0.266,0.238,131.121,4,dance


In [22]:
df[df.artists.str.contains("ENH|BTS")]

,Unnamed: 0,track_id,artists,album_name,track_name,popularity,duration_ms,explicit,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature,track_genre
4411,5007,0UzymivvUH5s8z4PeWZJaK,ENHYPEN,BORDER : CARNIVAL,FEVER,77,172106,False,0.600,0.713,6,-5.079,0,0.0592,0.064200,0.000,0.1420,0.4710,72.999,4,anime
4426,5022,5elW2CKSoqjYoJ32AGDxf1,ENHYPEN,DIMENSION : ANSWER,Polaroid Love,78,184601,False,0.738,0.670,7,-3.766,0,0.0439,0.458000,0.000,0.3610,0.6510,138.055,4,anime
4431,5027,1wcr8DjnN59Awev8nnKpQ4,ENHYPEN,BORDER : CARNIVAL,Drunk-Dazed,75,193260,False,0.599,0.869,5,-5.174,0,0.0618,0.000707,0.000,0.0911,0.4530,132.889,4,anime
4432,5028,3nV2Yjy2aJjUl74p0tEfWf,ENHYPEN,Make the change,Make the change,61,187000,False,0.807,0.767,0,-3.643,1,0.0777,0.030800,0.000,0.2850,0.8550,110.994,4,anime
4434,5030,66wQlkJP6zHNOzRkyo5yZS,ENHYPEN,One In A Billion,One In A Billion,69,196392,False,0.719,0.952,8,-3.058,0,0.0674,0.197000,0.000,0.1400,0.4460,119.957,4,anime
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
55888,65990,1e45vYKAMljTZUQUotLdq2,BTS,Proof,Intro : Persona,62,171023,False,0.486,0.871,9,-4.475,0,0.3370,0.027800,0.000,0.3130,0.6590,85.411,4,k-pop
67984,81882,4TgxFMOn5yoESW6zCidCXL,Jawsh 685;Jason Derulo;BTS,Savage Love (Laxed - Siren Beat) [BTS Remix],Savage Love (Laxed – Siren Beat) [BTS Remix],73,184905,True,0.758,0.598,0,-5.695,0,0.0868,0.160000,0.000,0.2950,0.6580,149.836,4,pop
69373,83653,1EXENmPTpn9d2JBJJKZnmQ,Lane 8;RBBTS,Visions,Visions,47,261120,False,0.516,0.888,8,-10.282,1,0.0328,0.182000,0.766,0.0983,0.0394,124.989,4,progressive-house
82420,105102,1gr9Risxpt6BfHdu5VxtAx,SLKBTS,sunday am,sunday am,43,118513,False,0.718,0.646,8,-8.211,0,0.2750,0.457000,0.234,0.1790,0.8120,80.233,4,study


In [23]:
df.describe()

,Unnamed: 0,popularity,duration_ms,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature
count,89740.000000,89740.000000,8.974000e+04,89740.000000,89740.000000,89740.000000,89740.000000,89740.000000,89740.000000,89740.000000,89740.000000,89740.000000,89740.000000,89740.000000,89740.000000
mean,53479.005739,33.198808,2.291444e+05,0.562166,0.634458,5.283530,-8.498994,0.636973,0.087442,0.328285,0.173415,0.216971,0.469474,122.058134,3.897426
std,33410.141924,20.580640,1.129458e+05,0.176692,0.256606,3.559912,5.221518,0.480875,0.113278,0.338321,0.323849,0.194885,0.262864,30.117651,0.453437
min,0.000000,0.000000,8.586000e+03,0.000000,0.000000,0.000000,-49.531000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,23766.750000,19.000000,1.730400e+05,0.450000,0.457000,2.000000,-10.322250,0.000000,0.036000,0.017100,0.000000,0.098200,0.249000,99.262750,4.000000
50%,50680.500000,33.000000,2.132955e+05,0.576000,0.676000,5.000000,-7.185000,1.000000,0.048900,0.188000,0.000058,0.132000,0.457000,122.013000,4.000000
75%,80618.500000,49.000000,2.642930e+05,0.692000,0.853000,8.000000,-5.108000,1.000000,0.085900,0.625000,0.097625,0.279000,0.682000,140.077000,4.000000
max,113999.000000,100.000000,5.237295e+06,0.985000,1.000000,11.000000,4.532000,1.000000,0.965000,0.996000,1.000000,1.000000,0.995000,243.372000,5.000000


In [24]:
numerical_cols = ['danceability', 'energy', 'loudness', 'speechiness', 
            'acousticness', 'instrumentalness', 'liveness', 'valence', 
            'tempo', 'popularity']

In [25]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
df_num = scaler.fit_transform(df[numerical_cols])

In [26]:
str_cols = ["artists", "album_name", "track_name", "track_genre"]

df[str_cols] = df[str_cols].astype("string")


In [27]:
df.iloc[2,2].strip().split(";")

['Ingrid Michaelson', 'ZAYN']

In [28]:
'Ingrid Michaelson'.replace(" ","")

'IngridMichaelson'

In [29]:
def jaccard_similarity(set1, set2):
    return len(set1 & set2) / len(set1 | set2)

In [30]:
df['artist_set'] = df['artists'].apply(lambda x: set([i.replace(" ","") for i in x.strip().lower().split(';')]))

In [31]:
from sklearn.metrics.pairwise import cosine_similarity
cosine_similarity(df_num[0].reshape(1,-1),df_num[1].reshape(1,-1))

array([[0.72365277]])

In [32]:
def similarity_score(id1,id2,numerical_weight=0.4,artist_weight=0.35,genre_weight=0.25):
    numerical_score=cosine_similarity(df_num[id1].reshape(1,-1),df_num[id2].reshape(1,-1))[0][0]
    
    artist_score=jaccard_similarity(df['artist_set'][id1],df['artist_set'][id2])
    
    genre_score=1 if df['track_genre'][id1]==df['track_genre'][id2] else 0
    
    return numerical_weight*numerical_score + artist_weight*artist_score + genre_weight* genre_score
    

In [33]:
similarity_score(4434,55888)

np.float64(0.38248255240248086)

In [34]:
df[df.track_name=="Lover"]

,Unnamed: 0,track_id,artists,album_name,track_name,popularity,duration_ms,explicit,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature,track_genre,artist_set
48090,54879,5kBC3lWxOG0393nUcK6lmX,Luke Vibert,Luke Vibert presents Rave Hop,Lover,10,253888,False,0.670,0.926,1,-4.776,0,0.0365,0.02520,0.169000,0.1960,0.659,108.008,4,idm,{lukevibert}
57904,68862,4rUQPct3yT97OXelj85R5Q,Young Cister,Lover,Lover,40,150563,False,0.763,0.533,6,-11.012,0,0.0489,0.77900,0.000073,0.1820,0.597,145.004,4,latino,{youngcister}
66806,80004,0XwRlvv3KlOu4HWlOH34XG,Diljit Dosanjh,MoonChild Era,Lover,71,190000,False,0.827,0.587,9,-5.531,0,0.1130,0.20300,0.000000,0.0727,0.774,119.936,4,pop-film,{diljitdosanjh}
67648,81091,6X7pjnNm5dHSUot6hp3KFU,Taylor Swift,Autumn Vibes 2022,Lover,0,221306,False,0.359,0.543,7,-7.582,1,0.0915,0.49200,0.000016,0.1180,0.453,68.505,4,pop,{taylorswift}
68023,81973,1dGr1c8CrMLDpV6mPbImSI,Taylor Swift,Lover,Lover,85,221306,False,0.359,0.543,7,-7.582,1,0.0919,0.49200,0.000016,0.1180,0.453,68.534,4,pop,{taylorswift}
69480,83781,40u9pTla24PIkyRLKKVh6a,Otto Knows;Dice Of Nights;Alex Aris,Lover,Lover,54,215580,False,0.659,0.873,2,-5.719,0,0.0773,0.00701,0.846000,0.1130,0.271,124.982,4,progressive-house,"{diceofnights, ottoknows, alexaris}"


In [35]:
similarity_score(68023,67984)

np.float64(0.6181918794612404)

In [36]:
def recommend(track_idx, top_n=10):
    input_track_name = df.iloc[track_idx]["track_name"]
    scores = np.array([similarity_score(track_idx, i) for i in range(len(df))])
    top_indices = scores.argsort()[::-1][:top_n]
    results=df.iloc[top_indices][['track_name','artists','album_name','track_genre']]
    results=results[results["track_name"]!=input_track_name]
    return results[:5]


In [37]:
df.loc[81937,:]

Unnamed: 0                             104616
track_id               4SafR3a8senZ2r65hZWMKN
artists                  El Último De La Fila
album_name          Insurreccion (En Directo)
track_name          Insurreccion (En Directo)
popularity                                 59
duration_ms                            134200
explicit                                False
danceability                            0.491
energy                                  0.954
key                                         0
loudness                               -4.007
mode                                        1
speechiness                            0.0881
acousticness                            0.112
instrumentalness                     0.000282
liveness                                 0.98
valence                                 0.606
tempo                                 136.198
time_signature                              4
track_genre                           spanish
artist_set                 {elúlti

In [38]:
recommend(81937)

,track_name,artists,album_name,track_genre
82148,Cuando el Mar te Tenga - En Directo,El Último De La Fila,Historia de una Banda (En Directo),spanish
81717,Como un burro amarrado en la puerta del baile ...,El Último De La Fila,"Las 100 mejores canciones del Pop Español, Vol. 3",spanish
81915,Insurrección - 2008 Remaster,El Último De La Fila,Enemigos de lo ajeno (2008 Remaster),spanish
81998,Aviones Plateados - En Directo,El Último De La Fila,Historia de una Banda (En Directo),spanish
82287,Nada Fue Un Error - Live In Spain / 2005,Coti;Paulina Rubio;Julieta Venegas,Básicos,spanish


In [39]:
recommend(55024)

,track_name,artists,album_name,track_genre
55404,Fly To My Room,BTS,BE,k-pop
54993,Dynamite,BTS,BE,k-pop
55206,Go Go,BTS,Love Yourself 承 'Her',k-pop
55188,My Time,BTS,MAP OF THE SOUL : 7,k-pop
55316,Stay Gold,BTS,MAP OF THE SOUL : 7 ~ THE JOURNEY ~,k-pop


In [40]:
recommend(55380)

,track_name,artists,album_name,track_genre
55161,Blue & Grey,BTS,BE,k-pop
55125,Epiphany,BTS,Love Yourself 結 'Answer',k-pop
55327,Answer : Love Myself,BTS,Love Yourself 結 'Answer',k-pop
55006,Euphoria,BTS,Love Yourself 結 'Answer',k-pop
55149,Serendipity - Full Length Edition,BTS,Love Yourself 結 'Answer',k-pop


In [41]:
import time
start=time.time()
rec=recommend(68023)
end=time.time()
print(f"{(end-start):.2f} sec")

43.12 sec


In [42]:
rec

,track_name,artists,album_name,track_genre
67866,august,Taylor Swift,folklore,pop
67674,Blank Space,Taylor Swift,1989 (Deluxe),pop
67997,Love Story,Taylor Swift,Fearless (Big Machine Radio Release Special),pop
67884,Wildest Dreams,Taylor Swift,1989,pop
67975,Don’t Blame Me,Taylor Swift,reputation,pop


- The above method is taking approximately 71 sec
- So let us see the results if we use TfIdfVectorizer and compare both results


In [43]:
import re

def string_info(row):
   
    artists = row['artists'].strip().split(";")
    artists = [a.replace(" ", "") for a in artists]
    
    
    #album = re.sub(r"[^a-zA-Z0-9 ]", "", row['album_name']).strip()  #removed this because of the model's high bias towards album name
    
    
    genre = row['track_genre'].replace(" ", "")
    
    
    exp_tag = "explicit" if row["explicit"] else "clean"
    
    
    info_list = artists + [genre, exp_tag]
    return " ".join(info_list).lower()

df['info'] = df.apply(string_info, axis=1)


In [44]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf=TfidfVectorizer(max_features=2000,stop_words='english')
vectors = tfidf.fit_transform(df['info'])


In [45]:
vectors.shape

(89740, 2000)

In [46]:
def find_similarity(track_id):
    similarity=cosine_similarity(vectors[track_id].reshape(1,-1),vectors)
    return similarity

In [47]:
list(enumerate(find_similarity(1)[0].tolist()))

[(0, 1.0),
 (1, 1.0),
 (2, 1.0),
 (3, 0.5154927951245685),
 (4, 1.0),
 (5, 1.0),
 (6, 1.0),
 (7, 0.5479052882422035),
 (8, 0.5479052882422035),
 (9, 1.0),
 (10, 0.5116430963702201),
 (11, 0.5479052882422035),
 (12, 1.0),
 (13, 1.0),
 (14, 1.0),
 (15, 1.0),
 (16, 0.3916310363557314),
 (17, 0.3916310363557314),
 (18, 0.5479052882422035),
 (19, 0.5469426867040207),
 (20, 0.5479052882422035),
 (21, 0.5469426867040207),
 (22, 1.0),
 (23, 0.5479052882422035),
 (24, 0.5479052882422035),
 (25, 0.5479052882422035),
 (26, 0.5479052882422035),
 (27, 0.5479052882422035),
 (28, 0.5479052882422035),
 (29, 0.5479052882422035),
 (30, 0.5479052882422035),
 (31, 0.5479052882422035),
 (32, 1.0),
 (33, 0.4054592453405285),
 (34, 0.4054592453405285),
 (35, 0.4054592453405285),
 (36, 1.0),
 (37, 0.5154927951245685),
 (38, 0.5154927951245685),
 (39, 1.0),
 (40, 0.5095908186654152),
 (41, 0.5154927951245685),
 (42, 0.5154927951245685),
 (43, 0.5154927951245685),
 (44, 0.5154927951245685),
 (45, 0.515492795124

In [48]:
def recommend_tfidf(track_name,artist=None):
    if artist:
        track_id=df[(df.track_name==track_name) & (df.artists.str.contains(artist))].index[0]
    else:
        track_id=df[df.track_name==track_name].index[0]
    sim=find_similarity(track_id)[0]
    top_5=sorted(list(enumerate(sim.tolist())),reverse=True,key=lambda x:x[1])[:10]
    ids=[id[0] for id in top_5]
    results=df.loc[ids,['track_id','track_name','artists','album_name']]
    results=results[results['track_name']!=track_name]
    return results[:5]

In [49]:
recommend_tfidf("Ghost - Acoustic")

,track_id,track_name,artists,album_name
0,5SuOikwiRyPMVoIQDJUgSV,Comedy,Gen Hoshino,Comedy
2,1iJBSr7s7jYXzM8EGcbK5b,To Begin Again,Ingrid Michaelson;ZAYN,To Begin Again
4,5vjLSffimiIP26QG5WcN2K,Hold On,Chord Overstreet,Hold On
5,01MVOl9KtVTNfFiBU9I7dc,Days I Will Remember,Tyrone Wells,Days I Will Remember
6,6Vc5wAMmXdKIAM7WUoEb7N,Say Something,A Great Big World;Christina Aguilera,Is There Anybody Out There?


In [50]:
recommend_tfidf("Perfect","Ed Sheeran")

,track_id,track_name,artists,album_name
67628,7qiZfU4dY1lWllzX7mPBI3,Shape of You,Ed Sheeran,÷ (Deluxe)
67719,7s2OAgMDtuwq8F3DaBhunU,Autumn Leaves,Ed Sheeran,Music for Rainy Days
67721,1euRAecw6xEwkt6lPXMbFP,Shivers,Ed Sheeran,Today's Party Hits
67723,4JWlQ6skcm0SuztTiapWeo,How Would You Feel (Paean),Ed Sheeran,Coffee Moment
67725,0MbP1Nr6DLXB7HFXD7QVwU,Visiting Hours,Ed Sheeran,20's Rock


In [51]:
recommend_tfidf("Lover","Taylor Swift")

,track_id,track_name,artists,album_name
67647,6vJaXvEXh6HOeiEZLFDhrp,You Belong With Me (Taylor’s Version),Taylor Swift,Sunny Mornings
67650,3WRr1TGp0glkYj7hL4unH5,You Belong With Me (Taylor’s Version),Taylor Swift,Autumn Vibes 2022
67651,3AYtDDIbG4DQxnEFELeFag,willow,Taylor Swift,Autumn Vibes 2022
67652,14MGlCeQzAJj3EeznSGeEC,cardigan,Taylor Swift,Autumn Vibes 2022
67674,1u8c2t2Cy7UBoG4ArRcF5g,Blank Space,Taylor Swift,1989 (Deluxe)


In [52]:
recommend_tfidf("Cuando el Mar te Tenga - En Directo")

,track_id,track_name,artists,album_name
81377,5ZEQjTroORCu6uWvZrdeNc,My Life Is Going On (Música Original De La Ser...,Cecilia Krull,My Life Is Going On (Música Original De La Ser...
81383,5SfpLM5B1l3CKDTYdbFhKV,Tiramisu de Limon,Joaquín Sabina,Éxitos del Pop en Español de los Años 2000
81387,3eKQ99bghyGg0Zx5sVWBEP,Tu Boca,Antonio José,Dulce Balada
81389,0baa96SpfaI20kBcI3bC65,Contigo,Antonio José,Dulce Balada
81390,10Q9usWXIWvSFFtE3rdcVW,Contigo,Antonio José,Retro Baladas


In [53]:
df[df.track_name.str.contains("Butter")]

,Unnamed: 0,track_id,artists,album_name,track_name,popularity,duration_ms,explicit,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature,track_genre,artist_set,info
522,522,4XlevPQOF6GkZCS2yWOkNn,Kina Grannis,Build Me Up Buttercup,Build Me Up Buttercup,48,166320,False,0.721,0.0963,0,-15.973,1,0.0502,0.945000,0.000001,0.1330,0.5760,124.868,4,acoustic,{kinagrannis},kinagrannis acoustic clean
2076,2077,1Vl9pXCmq6m2FbpxqHptTY,The Smashing Pumpkins,Rock & Halloween,Bullet With Butterfly Wings - Remastered 2012,2,257812,False,0.395,0.7510,10,-8.547,0,0.0618,0.000057,0.838000,0.0725,0.4550,121.561,4,alt-rock,{thesmashingpumpkins},thesmashingpumpkins alt-rock clean
2602,2604,4BggEwLhGfrbrl7JBhC8EC,Crazy Town,The Gift Of Game,Butterfly,73,216733,False,0.736,0.8110,9,-4.170,0,0.0810,0.001320,0.000142,0.1070,0.6090,103.502,4,alt-rock,{crazytown},crazytown alt-rock clean
3628,4223,6Rzit6BsuduWO2hUshV1Co,Tony Anderson,Butterflies (Snowfall Remixes),Butterflies - Piano Sonata,53,240000,False,0.313,0.0288,4,-28.271,0,0.0395,0.992000,0.911000,0.0779,0.0764,126.866,4,ambient,{tonyanderson},tonyanderson ambient clean
4031,4626,7cpVPjdyGplWZLlg66KuWQ,Dario Marianelli;Jean-Yves Thibaudet,Pride and Prejudice - OST,"Stars and Butterflies - From ""Pride & Prejudic...",50,118986,False,0.178,0.0135,7,-28.203,1,0.0460,0.964000,0.920000,0.1160,0.0392,47.370,4,ambient,"{dariomarianelli, jean-yvesthibaudet}",dariomarianelli jean-yvesthibaudet ambient clean
6506,7111,1EC4aQXz8KKzCzK6xuEivP,Hot Buttered Rum,Well-Oiled Machine,Guns Or Butter,29,212826,False,0.810,0.4220,2,-9.278,0,0.0593,0.815000,0.000015,0.0872,0.8020,120.072,4,bluegrass,{hotbutteredrum},hotbutteredrum bluegrass clean
13120,13958,7KRqUYQCU51iFx0gmI9Luc,Paul Johnson,Let Me See You Butterfly,Let Me See You Butterfly - Them Jeans Remix,5,310799,False,0.810,0.8400,0,-6.616,1,0.0442,0.005510,0.000090,0.0459,0.7700,123.994,4,chicago-house,{pauljohnson},pauljohnson chicago-house clean
13121,13959,0YPBQ5feEP0G0hdBJIJkjF,Paul Johnson,Let Me See You Butterfly,Let Me See You Butterfly,6,311649,False,0.793,0.9050,1,-7.258,1,0.1240,0.000199,0.941000,0.0291,0.4730,132.796,4,chicago-house,{pauljohnson},pauljohnson chicago-house clean
13608,14454,1Vwa3ALXUjCpPkZ5fqeSHK,Super Simple Songs,Beddy-Bye Butterfly,Beddy-Bye Butterfly,53,197691,False,0.618,0.1740,10,-9.408,1,0.0296,0.967000,0.000000,0.3370,0.4680,76.222,4,children,{supersimplesongs},supersimplesongs children clean
14483,15354,6mCljAhAwsKTCsdaA0tIYF,ASTN,Butterflies,Butterflies,61,175225,False,0.858,0.3590,8,-12.197,1,0.0437,0.146000,0.000004,0.1010,0.4320,99.989,4,chill,{astn},astn chill clean


In [54]:
df[df.track_name.str.contains("Fools")]

,Unnamed: 0,track_id,artists,album_name,track_name,popularity,duration_ms,explicit,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature,track_genre,artist_set,info
21295,22544,385eS54sQTs5vG5NkU9L6l,Evil Darkness,Dark Blasting Tongue,Fools Bastards,23,82466,True,0.398,0.810,3,-9.322,0,0.0656,0.031700,0.835,0.319,0.694,72.658,4,death-metal,{evildarkness},evildarkness death-metal explicit
37656,42201,32FKk3um0YmeMxgGIBDD5v,P.L.F.,Devious Persecution and Wholesale Slaughter,Grinder of Fools,16,106700,False,0.255,0.996,10,-4.790,1,0.1370,0.000001,0.851,0.215,0.112,110.064,4,grindcore,{p.l.f.},p.l.f. grindcore clean


In [55]:
df[df.artists.str.contains("Troye Sivan")]

,Unnamed: 0,track_id,artists,album_name,track_name,popularity,duration_ms,explicit,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature,track_genre,artist_set,info
18236,19277,0lKZAkE4ni8ruTOPli9F5k,Kacey Musgraves;Troye Sivan,Best Christmas Playlist Ever 2022,Glittery - From The Kacey Musgraves Christmas ...,0,143596,False,0.595,0.275,8,-9.815,0,0.0309,0.8640,0.000000,0.1070,0.345,85.752,4,country,"{troyesivan, kaceymusgraves}",kaceymusgraves troyesivan country clean
18242,19283,7CApeRUBX1cqQHmyCC1ofY,Kacey Musgraves;Troye Sivan,The Very Best Christmas Songs 2022,Glittery - From The Kacey Musgraves Christmas ...,0,143596,False,0.595,0.275,8,-9.815,0,0.0309,0.8640,0.000000,0.1070,0.345,85.752,4,country,"{troyesivan, kaceymusgraves}",kaceymusgraves troyesivan country clean
18911,20006,3afkJSKX0EAMsJXTZnDXXJ,Troye Sivan,Strawberries & Cigarettes,Strawberries & Cigarettes,82,201538,False,0.578,0.537,9,-8.676,1,0.0832,0.1620,0.000000,0.0760,0.250,83.909,4,dance,{troyesivan},troyesivan dance clean
18959,20054,2m6Ko3CY1qXNNja8AlugNc,Troye Sivan,Angel Baby,Angel Baby,82,220779,False,0.559,0.559,11,-6.425,1,0.0358,0.0145,0.000000,0.1380,0.338,72.498,4,dance,{troyesivan},troyesivan dance clean
19297,20408,4vjpLMyL3ZO6aphdYr8XYw,Lauv;Troye Sivan,~how i'm feeling~,i'm so tired...,76,162586,False,0.576,0.736,11,-7.572,1,0.2240,0.1780,0.000000,0.2380,0.484,101.928,4,dance,"{lauv, troyesivan}",lauv troyesivan dance clean
19782,20911,0nhBKubnVz9yFNNprBniWz,Troye Sivan,Blue Neighbourhood (Deluxe),YOUTH,72,185194,False,0.628,0.737,7,-4.437,1,0.0410,0.0625,0.000000,0.0777,0.592,91.507,4,dance,{troyesivan},troyesivan dance clean
27730,30200,2cc8Sw1OnCuA5bV8nqWqpE,Regard;Troye Sivan;Tate McRae,You,You,76,233386,True,0.691,0.695,0,-5.600,1,0.0367,0.0592,0.000000,0.0647,0.514,106.064,4,edm,"{troyesivan, regard, tatemcrae}",regard troyesivan tatemcrae edm explicit
27731,30203,6jA8HL9i4QGzsj6fjoxp8Y,Martin Garrix;Troye Sivan,There for You,There for You,69,221904,False,0.611,0.644,6,-7.607,0,0.0553,0.1240,0.000000,0.1240,0.130,105.969,4,edm,"{troyesivan, martingarrix}",martingarrix troyesivan edm clean
28592,31337,1ZDzahk1OMKP9poJJ3VVpu,Charli XCX;Troye Sivan,Am I Ready - Pop Night Out,1999,3,189000,False,0.741,0.743,7,-4.586,1,0.0328,0.0224,0.000001,0.2290,0.658,124.014,4,electro,"{troyesivan, charlixcx}",charlixcx troyesivan electro clean
69208,83460,5K3NDUPkLe46b5DRMt2RNJ,Martin Garrix;Troye Sivan;Madison Mars,There For You: The Remixes,There For You - Madison Mars Remix,49,175000,False,0.600,0.947,6,-1.840,0,0.1630,0.0395,0.000000,0.1000,0.347,126.045,4,progressive-house,"{troyesivan, martingarrix, madisonmars}",martingarrix troyesivan madisonmars progressiv...


In [56]:
recommend_tfidf("Butter")

,track_id,track_name,artists,album_name
54990,5FVbvttjEvQ8r2BgUcJgNg,Life Goes On,BTS,BE
54993,5QDLhrAOJJdNAmCTJ8xMyW,Dynamite,BTS,BE
55000,6m1TWFMeon7ai9XLOzdbiR,FAKE LOVE,BTS,Love Yourself 轉 'Tear'
55006,5YMXGBD6vcYP7IolemyLtK,Euphoria,BTS,Love Yourself 結 'Answer'
55012,69xohKu8C1fsflYAiSNbwM,Run BTS,BTS,Proof


- Comparision of both approaches and limitations of TfIdf--------------?

In [76]:
recommend_tfidf("FAKE LOVE")

,track_id,track_name,artists,album_name
54990,5FVbvttjEvQ8r2BgUcJgNg,Life Goes On,BTS,BE
54993,5QDLhrAOJJdNAmCTJ8xMyW,Dynamite,BTS,BE
55006,5YMXGBD6vcYP7IolemyLtK,Euphoria,BTS,Love Yourself 結 'Answer'
55012,69xohKu8C1fsflYAiSNbwM,Run BTS,BTS,Proof
55013,10SRMwb9EuVS1K9rYsBfHQ,Yet To Come,BTS,Proof


In [77]:
recommend_tfidf("00:00 (Zero O’Clock)")

,track_id,track_name,artists,album_name
54990,5FVbvttjEvQ8r2BgUcJgNg,Life Goes On,BTS,BE
54993,5QDLhrAOJJdNAmCTJ8xMyW,Dynamite,BTS,BE
55000,6m1TWFMeon7ai9XLOzdbiR,FAKE LOVE,BTS,Love Yourself 轉 'Tear'
55006,5YMXGBD6vcYP7IolemyLtK,Euphoria,BTS,Love Yourself 結 'Answer'
55012,69xohKu8C1fsflYAiSNbwM,Run BTS,BTS,Proof


In [59]:
recommend_tfidf("Run BTS")

,track_id,track_name,artists,album_name
54990,5FVbvttjEvQ8r2BgUcJgNg,Life Goes On,BTS,BE
54993,5QDLhrAOJJdNAmCTJ8xMyW,Dynamite,BTS,BE
55000,6m1TWFMeon7ai9XLOzdbiR,FAKE LOVE,BTS,Love Yourself 轉 'Tear'
55006,5YMXGBD6vcYP7IolemyLtK,Euphoria,BTS,Love Yourself 結 'Answer'
55013,10SRMwb9EuVS1K9rYsBfHQ,Yet To Come,BTS,Proof


In [78]:
recommend_tfidf("YOUTH")

,track_id,track_name,artists,album_name
18911,3afkJSKX0EAMsJXTZnDXXJ,Strawberries & Cigarettes,Troye Sivan,Strawberries & Cigarettes
18927,0qOnSQQF0yzuPWsXrQ9paz,Stereo Hearts (feat. Adam Levine),Gym Class Heroes;Adam Levine,The Papercut Chronicles II
18958,4hyVrAsoKKjxAvQjPRt0ai,Love Your Voice,JONY,Список твоих мыслей
18959,2m6Ko3CY1qXNNja8AlugNc,Angel Baby,Troye Sivan,Angel Baby
19008,3C0nOe05EIt1390bVABLyN,On The Floor,Jennifer Lopez;Pitbull,Love?


- Now I will modify the recommend_tfidf function by adding some weightage to numerical_features and do some modifications and hopefully the results comes out better

In [80]:
def recommend_tfidf_v2(track_name,artist=None, num_weight=0.6, tfidf_weight=0.4):
    if artist:
        track_id=df[(df.track_name==track_name) & (df.artists.str.contains(artist))].index[0]
    else:
        track_id=df[df.track_name==track_name].index[0]
    tfidf_sim=find_similarity(track_id)[0]
    num_sim = cosine_similarity(df_num[track_id].reshape(1, -1), df_num)[0]
    
    sim = num_weight * num_sim + tfidf_weight * tfidf_sim
    
    top_5=sorted(list(enumerate(sim.tolist())),reverse=True,key=lambda x:x[1])[:10]
    ids=[id[0] for id in top_5]
    results=df.loc[ids,['track_id','track_name','artists','album_name']]
    results=results[results['track_name']!=track_name]
    results = results.drop_duplicates(subset=['track_name'])
    
    return results[:5]

In [81]:
recommend_tfidf_v2("Butterfly","BTS")

,track_id,track_name,artists,album_name
55633,0bGJHUPR6ems9tzIljlPFR,Begin,BTS,Wings
55123,5MTGPkmiUuud9NkA0sl2nI,Magic Shop,BTS,Love Yourself 轉 'Tear'
55264,25MHcbjvSdcfTiFgbKJiZF,Inner Child,BTS,MAP OF THE SOUL : 7
55418,0TdWVbcerMAcvljXrl9gpG,Stay,BTS,BE
55606,3YspylwDrs1LuzPONbKmAL,FAKE LOVE,BTS,Proof


In [82]:
def find_details(track_name=None,artist=None): 
    if track_name: 
        return df[df.track_name.str.contains(track_name)] 
    else: 
        return df[df.artists.str.contains(artist)]

In [83]:
find_details("Winter Bear")

,Unnamed: 0,track_id,artists,album_name,track_name,popularity,duration_ms,explicit,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature,track_genre,artist_set,info
66255,79330,3eTbOx5tsMwmdrEAVJkR5z,Smyang Piano,Playroom,Winter Bear,41,146666,False,0.416,0.395,7,-13.216,1,0.0289,0.929,0.926,0.106,0.0889,161.931,3,piano,{smyangpiano},smyangpiano piano clean


In [95]:
recommend_tfidf("Winter Bear")

,track_id,track_name,artists,album_name
66086,3YhzWdoDeIWvxtj8h4zrMl,Still With You,Smyang Piano,Still With You
66185,3eabrKx1t8T4sHLEAMmOXK,Euphoria,Smyang Piano,"Piano Collection, Vol. 4"
66187,11NT523llsajdTS0yVddPR,Spring Day,Smyang Piano,"Piano Collection, Vol. 1"
66251,79JqhhT62bJSWP7OqQPlnc,Euphoria - Lofi Edit,Smyang Piano;tamagopalette,No Hard Feelings
66254,1FJswUXEo8jr5E1zLjjClr,Intro: Serendipity,Smyang Piano,"Piano Collection, Vol. 1"


In [94]:
recommend_tfidf_v2("Lover",artist="Taylor Swift")

,track_id,track_name,artists,album_name
67651,3AYtDDIbG4DQxnEFELeFag,willow,Taylor Swift,Autumn Vibes 2022
67652,14MGlCeQzAJj3EeznSGeEC,cardigan,Taylor Swift,Autumn Vibes 2022
67650,3WRr1TGp0glkYj7hL4unH5,You Belong With Me (Taylor’s Version),Taylor Swift,Autumn Vibes 2022
67866,3hUxzQpSfdDqwM3ZTFQY0K,august,Taylor Swift,folklore
67997,1D4PL9B8gOg78jiHg3FvBb,Love Story,Taylor Swift,Fearless (Big Machine Radio Release Special)


In [86]:
find_details("Thodi Jagah")

,Unnamed: 0,track_id,artists,album_name,track_name,popularity,duration_ms,explicit,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature,track_genre,artist_set,info
66898,80117,3f0wz6zcA7XkQiGm0qNF5r,Arijit Singh;Tanishk Bagchi,"Thodi Jagah (From ""Marjaavaan"")","Thodi Jagah (From ""Marjaavaan"")",67,218048,False,0.621,0.467,0,-4.835,0,0.0273,0.632,0.0,0.0888,0.227,107.164,4,pop-film,"{tanishkbagchi, arijitsingh}",arijitsingh tanishkbagchi pop-film clean


In [93]:
recommend_tfidf_v2('Thodi Jagah (From "Marjaavaan")')

,track_id,track_name,artists,album_name
67067,68WsICkbiK1IOpDEQMdrjB,"Mahi Mera Dil (From ""Dhokha Round D Corner"")",Arijit Singh;Tulsi Kumar;Tanishk Bagchi;Kumaar,"Mahi Mera Dil (From ""Dhokha Round D Corner"")"
66921,2PvaburAUlhNHxVhw5lvOq,"Humdard (From ""Ek Villain"")",Arijit Singh,Ultimate Love Songs - Arijit Singh
66890,1OEmd2iXU8pWG7CrKNpGUg,Humdard,Arijit Singh,Ek Villain
67100,06zJUeKU5KFy50Cho6pDcr,Aasan Nahin Yahan,Arijit Singh,Aashiqui 2
66803,6pzlnyHTVYi5FOmrgOmC95,Jaan 'Nisaar (Arijit),Arijit Singh,Kedarnath


In [71]:
find_details('Vennelav')

,Unnamed: 0,track_id,artists,album_name,track_name,popularity,duration_ms,explicit,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature,track_genre,artist_set,info
67442,80797,08Zh9GFP2TKItpgcfRSHUV,Hariharan;Sadhana Sargam,Merupu Kalalu,Vennelave Vennelave,57,359773,False,0.665,0.19,6,-17.811,1,0.0339,0.497,0.000001,0.158,0.54,91.077,4,pop-film,"{sadhanasargam, hariharan}",hariharan sadhanasargam pop-film clean


In [92]:
recommend_tfidf_v2("Vennelave Vennelave")

,track_id,track_name,artists,album_name
67162,5xVGxxFjjFsyxGpXE0U8Qt,"Vennilavae (From ""Minsara Kanavu"")",Hariharan;Sadhana Sargam,Hits Of Vairamuthu
67227,3hp56aHySTDt9bdZiwdDO1,Pachai Nirame,Hariharan;Clinton Cerejo,Alaipayuthey
67452,0ZGicTeWpoIMpJax3NAuhn,Hai Rama,Hariharan;Swarnalatha,Rangeela (Original Motion Picture Soundtrack)
67569,5eQYzdcEDOhvJ4ClsUQB3C,Pachchadanamey,Hariharan;Clinton Cerejo,Sakhi
67503,6nwRC0UkS1MdOOMjcJsrbX,Tu Hi Re - Bombay / Soundtrack Version,Hariharan;Kavita Krishnamurthy,Bombay


In [91]:
recommend_tfidf_v2("Tu Hi Re - Bombay / Soundtrack Version")

,track_id,track_name,artists,album_name
67452,0ZGicTeWpoIMpJax3NAuhn,Hai Rama,Hariharan;Swarnalatha,Rangeela (Original Motion Picture Soundtrack)
67162,5xVGxxFjjFsyxGpXE0U8Qt,"Vennilavae (From ""Minsara Kanavu"")",Hariharan;Sadhana Sargam,Hits Of Vairamuthu
67442,08Zh9GFP2TKItpgcfRSHUV,Vennelave Vennelave,Hariharan;Sadhana Sargam,Merupu Kalalu
67569,5eQYzdcEDOhvJ4ClsUQB3C,Pachchadanamey,Hariharan;Clinton Cerejo,Sakhi
67227,3hp56aHySTDt9bdZiwdDO1,Pachai Nirame,Hariharan;Clinton Cerejo,Alaipayuthey


In [73]:
recommend_tfidf("YOUTH")

,track_id,track_name,artists,album_name
18911,3afkJSKX0EAMsJXTZnDXXJ,Strawberries & Cigarettes,Troye Sivan,Strawberries & Cigarettes
18927,0qOnSQQF0yzuPWsXrQ9paz,Stereo Hearts (feat. Adam Levine),Gym Class Heroes;Adam Levine,The Papercut Chronicles II
18958,4hyVrAsoKKjxAvQjPRt0ai,Love Your Voice,JONY,Список твоих мыслей
18959,2m6Ko3CY1qXNNja8AlugNc,Angel Baby,Troye Sivan,Angel Baby
19008,3C0nOe05EIt1390bVABLyN,On The Floor,Jennifer Lopez;Pitbull,Love?


In [90]:
recommend_tfidf_v2("YOUTH")

,track_id,track_name,artists,album_name
19740,59tskctgqUmjCWAwhzYAFm,Looking at Me,Sabrina Carpenter,Singular Act II
19581,6T689Jvh5KrSXyaPtUWZtQ,Criminal,Britney Spears,Femme Fatale (Deluxe Version)
19008,3C0nOe05EIt1390bVABLyN,On The Floor,Jennifer Lopez;Pitbull,Love?
19625,6l0Ky2PycJV1Bz88kFeVGL,Capital Letters,Hailee Steinfeld;BloodPop®,Fifty Shades Freed (Original Motion Picture So...
19526,71R6zJsrF3ffc3TBFHfivX,Bumpy Ride,Mohombi,MoveMeant (International)


In [89]:
recommend_tfidf("HOT")

,track_id,track_name,artists,album_name
55119,1pdFdv8R6ezIAUUNkn785b,CHEERS,SEVENTEEN,SEVENTEEN 4th Album Repackage 'SECTOR 17'
55171,3QwiidVHfeE9y5jl4n2MTC,_WORLD,SEVENTEEN,SEVENTEEN 4th Album Repackage 'SECTOR 17'
55219,6LnEoRQKMcaFTR5UvaKuBy,Rock with you,SEVENTEEN,SEVENTEEN 9th Mini Album 'Attacca'
55366,1Rrj7KyS2R6SP9CQMDJW1w,VERY NICE,SEVENTEEN,Love&Letter repackage album
55390,6Upu6yjkdi0DVI8E3IBZEU,Don't Wanna Cry,SEVENTEEN,SEVENTEEN 4th Mini Album ‘Al1’


In [96]:
recommend_tfidf_v2("HOT")

,track_id,track_name,artists,album_name
55693,2FymmKBuog0loCuNXMwQID,Ready to love,SEVENTEEN,SEVENTEEN 8th Mini Album 'Your Choice'
55390,6Upu6yjkdi0DVI8E3IBZEU,Don't Wanna Cry,SEVENTEEN,SEVENTEEN 4th Mini Album ‘Al1’
55785,19t5GSN3XsLB7UOsZD8Fwv,CLAP,SEVENTEEN,"SEVENTEEN 2ND ALBUM 'TEEN, AGE' (2)"
55792,7BiEgR5r0yMmFoWC9kOTGf,DON QUIXOTE,SEVENTEEN,SEVENTEEN 4th Album 'Face the Sun'
55219,6LnEoRQKMcaFTR5UvaKuBy,Rock with you,SEVENTEEN,SEVENTEEN 9th Mini Album 'Attacca'
